In [1]:
import os
import sys
import polars as pl
import polars.selectors as cs

from datetime import datetime

PROJECT_PATH = os.path.abspath(os.path.join(".."))
PIPELINE_PATH = os.path.join(PROJECT_PATH, "pipeline")
DATA_DIRECTORY = os.path.join(PROJECT_PATH, "data")

unflagged_data = False

if PIPELINE_PATH not in sys.path:
    sys.path.append(PIPELINE_PATH)
    
from utils.paths import PROCESSED_PICARRO_DATA_DIRECTORY, POSTPROCESSED_DATA_DIRECTORY
from utils.import_data import import_acropolis_system_data
from utils.plot_dataframes import plot_sensor_measurement, plot_wind_rose, plot_co2_rose

assert(os.path.exists(POSTPROCESSED_DATA_DIRECTORY))
assert(os.path.exists(PROCESSED_PICARRO_DATA_DIRECTORY))

In [9]:
# Filters
start_date = datetime(2024, 3, 28, 0, 0, 0)
end_date = datetime(2024, 4, 17, 9, 59, 59)

filter = '1h'
ids = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

In [3]:
# Load Picarro Data
# DWD Picarro: "Calibrated_1_min_DWD_Picarro_G2301_413.parquet"
df_p_dwd = pl.scan_parquet(os.path.join(PROCESSED_PICARRO_DATA_DIRECTORY,"Calibrated_1_min_DWD_Picarro_G2301_413.parquet"))
df_p_dwd = df_p_dwd.filter(pl.col("datetime").is_between(start_date, end_date)).collect()

# ICOS Picarro: "Calibrated_1_min_ICOS_Picarro_G2401_529.parquet"
df_p_icos = pl.scan_parquet(os.path.join(PROCESSED_PICARRO_DATA_DIRECTORY,"Calibrated_1_min_ICOS_Picarro_G2401_529.parquet"))
df_p_icos = df_p_icos.filter(pl.col("datetime").is_between(start_date, end_date)).collect()
    
df_p = pl.concat([df_p_dwd,df_p_icos], how="diagonal")
del df_p_dwd, df_p_icos
df_p.head(1).vstack(df_p.tail(1))

system_name,system_id,datetime,picarro_corrected,h2o_reported
str,i32,datetime[ms],f64,f64
"""Picarro_G2301""",413,2024-03-28 00:00:00,440.480792,1.007596
"""Picarro_G2301""",413,2024-04-17 09:59:00,444.919328,1.067522


In [4]:
# Load ACROPOLIS Data
all_systems = []

for id in ids:
    df_system = import_acropolis_system_data(
            years=[2024,2025],
            target_directory=POSTPROCESSED_DATA_DIRECTORY,
            id=id,
            prefix="1min"
        ).group_by_dynamic("datetime", every=filter, group_by=["system_id", "system_name"]) \
            .agg(cs.numeric().mean()) \
        .filter(pl.col("datetime").is_between(start_date, end_date))
        
    all_systems.append(df_system)
    
del df_system
df = pl.concat(all_systems, how="diagonal").collect()
df.head(1).vstack(df.tail(1))

system_id,system_name,datetime,ts,gmp343_raw,gmp343_compensated,gmp343_filtered,gmp343_temperature,bme280_temperature,bme280_humidity,bme280_pressure,sht45_temperature,sht45_humidity,gmp343_edge_corrected,gmp343_edge_dry,h2o_ah,h2o_v%,bme280_h2o_v%,gmp343_dry,slope,intercept,slope_interpolated,intercept_interpolated,gmp343_corrected,wxt532_direction_min,wxt532_direction_avg,wxt532_direction_max,wxt532_speed_min,wxt532_speed_avg,wxt532_speed_max,wxt532_last_update_time,wxt532_temperature,wxt532_heating_voltage,wxt532_supply_voltage,wxt532_reference_voltage,enclosure_bme280_temperature,enclosure_bme280_humidity,enclosure_bme280_pressure,raspi_cpu_temperature,raspi_disk_usage,raspi_cpu_usage,raspi_memory_usage,ups_powered_by_grid,ups_battery_is_fully_charged,ups_battery_error_detected,ups_battery_above_voltage_threshold,cal_gmp343_slope,cal_gmp343_intercept,cal_sht_45_offset
i32,str,datetime[ms],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,"""tum-esm-midcost-raspi-1""",2024-03-28 00:00:00,1.7116e12,398.016556,473.3155,473.342167,31.532778,27.598289,17.834961,916.039717,27.583017,21.516378,null,null,7.092867,1.088777,0.902491,478.552515,0.977428,-30.149703,0.975394,-28.94724,437.830247,52.55,174.09,317.7,0.123333,1.778333,3.813333,1.7116e9,null,null,null,null,34.25,15.891333,931.149333,53.641667,0.507,0.020283,0.12215,1.0,1.0,0.0,1.0,null,null,null
20,"""tum-esm-midcost-raspi-20""",2024-04-17 09:00:00,1.7133e12,380.526667,444.045556,444.106389,32.256667,27.842389,16.214222,929.200556,28.479889,21.618861,null,null,7.406929,1.123545,0.842664,449.152855,0.993152,-10.88432,0.996188,-12.685834,434.75471,8.633333,288.456667,354.0,0.165,2.076667,4.218333,1.7133e9,null,null,null,null,34.900333,11.218167,943.201167,54.775,0.395,0.019017,0.112367,1.0,1.0,0.0,1.0,null,null,null


In [10]:
# Load CAL ACROPOLIS Data (no aggregation)
all_systems = []

for id in ids:
    df_system = import_acropolis_system_data(
            years=[2024,2025],
            target_directory=POSTPROCESSED_DATA_DIRECTORY,
            id=id,
            prefix="Cal_1min"
        ) \
        .filter(pl.col("datetime").is_between(start_date, end_date))
    
    all_systems.append(df_system)
    
del df_system
df_cal = pl.concat(all_systems, how="diagonal").collect()
df_cal.head(1).vstack(df_cal.tail(1))

datetime,system_id,cal_bottle_id,cal_gmp343_raw,cal_gmp343_compensated,cal_gmp343_filtered,cal_gmp343_temperature,cal_bme280_temperature,cal_bme280_humidity,cal_bme280_pressure,cal_sht45_temperature,cal_sht45_humidity,cal_gmp343_slope,cal_gmp343_intercept,cal_sht_45_offset
datetime[ms],i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2024-03-28 03:01:02.370,1,67.0,401.4,477.2,477.1,31.3,27.29,15.99,874.41,27.25,19.8,null,null,null
2024-04-16 03:46:35.250,20,46.0,361.2,419.3,420.7,33.4,35.88,0.0,938.42,36.57,1.46,null,null,null


In [ ]:
plot_sensor_measurement(df,ids,col_name="enclosure_bme280_humidity")

In [ ]:
plot_sensor_measurement(df,ids,col_name="gmp343_temperature")

In [ ]:
plot_sensor_measurement(df,ids=ids,col_name="gmp343_corrected")

In [ ]:
plot_sensor_measurement(df,ids=ids,col_name="wxt532_speed_avg")

In [ ]:
plot_sensor_measurement(df,ids,col_name="sht45_humidity")

In [ ]:
plot_sensor_measurement(df,ids,col_name="h2o_v%")

In [ ]:
plot_sensor_measurement(df,ids,col_name="bme280_humidity")

In [ ]:
plot_sensor_measurement(df,ids,col_name="bme280_h2o_v%")

In [ ]:
df_plot = df.with_columns(diff_inflow_humidity = (pl.col("sht45_humidity") - pl.col("bme280_humidity"))) \
    .filter(pl.col("diff_inflow_humidity").is_not_nan()) \
    .select("datetime", "diff_inflow_humidity", "system_id") \
    .sort("system_id")
    
plot_sensor_measurement(df_plot,ids,col_name="diff_inflow_humidity")

In [ ]:
df_plot = df.with_columns(diff_inflow_humidity = (pl.col("h2o_v%") - pl.col("bme280_h2o_v%"))) \
    .filter(pl.col("diff_inflow_humidity").is_not_nan()) \
    .select("datetime", "diff_inflow_humidity", "system_id") \
    .sort("system_id")
    
plot_sensor_measurement(df_plot,ids,col_name="diff_inflow_humidity")

In [ ]:
plot_sensor_measurement(df,ids,col_name="sht45_temperature")

In [ ]:
plot_sensor_measurement(df,ids,col_name="bme280_temperature")

In [ ]:
df_plot = df.with_columns(diff_inflow_temperature = (pl.col("sht45_temperature") - pl.col("bme280_temperature"))) \
    .filter(pl.col("diff_inflow_temperature").is_not_nan()) \
    .select("datetime", "diff_inflow_temperature", "system_id") \
    .sort("system_id")
    
plot_sensor_measurement(df_plot,ids,col_name="diff_inflow_temperature")

In [ ]:
plot_sensor_measurement(df,ids,col_name="bme280_pressure")

In [ ]:
plot_sensor_measurement(df,ids,col_name="enclosure_bme280_temperature")

In [ ]:
plot_sensor_measurement(df,ids=ids,col_name="gmp343_raw")

In [ ]:
plot_sensor_measurement(df,ids,col_name="gmp343_compensated")

In [ ]:
plot_sensor_measurement(df,ids,col_name="wxt532_temperature")

In [ ]:
# KLEG
plot_co2_rose(df, 1, 'Klenze Gymnasium')

# X
#plot_co2_rose(df, 2, 'X')

# X
#plot_co2_rose(df, 3, 'X')

# X
#plot_co2_rose(df, 4, 'X')

# DLRO
plot_co2_rose(df, 5, 'DLR')

# TUMN
plot_co2_rose(df, 6, 'TUM')

# FB85
#plot_co2_rose(df, 7, 'Blutenburgturm 85m')

# KRDI
plot_co2_rose(df, 8, 'Klinikum Rechts-der-Isar')

# KNPL
plot_co2_rose(df, 9, 'Klinikum Neuperlach')

# SWGG
plot_co2_rose(df, 10, 'Städtisches Willi Graf Gymnasium')

# RFIN
plot_co2_rose(df, 11, 'Rathaus Finsing')

# WKRT
plot_co2_rose(df, 12, 'Walter-Klingenbeck-Realschule')

# RFEL
plot_co2_rose(df, 13, 'Rathaus Feldkirchen')

# FB48
#plot_co2_rose(df, 14, 'Blutenburgturm 48m')

# SWMZ
plot_co2_rose(df, 15, 'Stadtwerke München Zentrale')

# SMAI
plot_co2_rose(df, 16, 'Schule Maisach')

# KBOG
#plot_co2_rose(df, 17, 'Klinikum Bogenhausen')

# SBBG
plot_co2_rose(df, 18, 'Städtisches Bertolt Brecht Gymnasium')

# X
#plot_o2_rose(df, 19, 'X')

# LGRO
plot_co2_rose(df, 20, 'LMU Großhadern')



In [ ]:
# KLEG
plot_wind_rose(df, 1, 'Klenze Gymnasium')

# X
#plot_wind_rose(df, 2, 'X')

# X
#plot_wind_rose(df, 3, 'X')

# X
#plot_wind_rose(df, 4, 'X')

# DLRO
plot_wind_rose(df, 5, 'DLR')

# TUMN
plot_wind_rose(df, 6, 'TUM')

# FB85
#plot_wind_rose(df, 7, 'Blutenburgturm 85m')

# KRDI
plot_wind_rose(df, 8, 'Klinikum Rechts-der-Isar')

# KNPL
plot_wind_rose(df, 9, 'Klinikum Neuperlach')

# SWGG
plot_wind_rose(df, 10, 'Städtisches Willi Graf Gymnasium')

# RFIN
plot_wind_rose(df, 11, 'Rathaus Finsing')

# WKRT
plot_wind_rose(df, 12, 'Walter-Klingenbeck-Realschule')

# RFEL
plot_wind_rose(df, 13, 'Rathaus Feldkirchen')

# FB48
#plot_wind_rose(df, 14, 'Blutenburgturm 48m')

# SWMZ
plot_wind_rose(df, 15, 'Stadtwerke München Zentrale')

# SMAI
plot_wind_rose(df, 16, 'Schule Maisach')

# KBOG
#plot_wind_rose(df, 17, 'Klinikum Bogenhausen')

# SBBG
plot_wind_rose(df, 18, 'Städtisches Bertolt Brecht Gymnasium')

# X
#plot_o2_rose(df, 19, 'X')

# LGRO
plot_wind_rose(df, 20, 'LMU Großhadern')



In [ ]:
plot_sensor_measurement(df,ids,col_name="wxt532_direction_avg")

In [ ]:
plot_sensor_measurement(df,ids,col_name="raspi_cpu_usage")

In [ ]:
plot_sensor_measurement(df,ids,col_name="raspi_memory_usage")


In [ ]:
plot_sensor_measurement(df,ids,col_name="raspi_disk_usage")


In [ ]:
plot_sensor_measurement(df,ids,col_name="raspi_cpu_temperature")

In [ ]:
plot_sensor_measurement(df,ids,col_name="enclosure_bme280_pressure")

In [ ]:
df_plot = df.with_columns(pl.col("enclosure_bme280_pressure").forward_fill().backward_fill()) \
    .with_columns(measurement_pressure = (pl.col("bme280_pressure") - pl.col("enclosure_bme280_pressure"))) \
    .filter(pl.col("measurement_pressure").is_not_nan()) \
    .select("datetime", "measurement_pressure", "system_id") \
    .sort("system_id")
    

import matplotlib.pyplot as plt
import polars as pl

for id in ids:
    
    # Filter data for the given system_id
    filtered_df = (
    df_plot.filter(pl.col("system_id") == id)
)

    # Plot using Matplotlib
    plt.figure(figsize=(10, 6))
    plt.plot(
        filtered_df["datetime"], 
        filtered_df["measurement_pressure"], 
        marker='o', 
        label=f"System {id}"
    )

    # Customize plot
    plt.title(f"Pressure Drop During Measurement for System {id}", fontsize=14)
    plt.xlabel("Timestamp", fontsize=12)
    plt.ylabel("measurement_pressure", fontsize=12)
    plt.xticks(rotation=45)
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(fontsize=12)

    # Show the plot
    plt.tight_layout()
    plt.show()

# Calibration Data

In [ ]:
plot_sensor_measurement(df=df_cal,sensor_id=ids,col_name="cal_bottle_id")

In [ ]:
plot_sensor_measurement(df_cal,ids,col_name="cal_gmp343_filtered")

In [13]:
plot_sensor_measurement(df_cal,[14],col_name="cal_sht45_humidity")

In [ ]:
plot_sensor_measurement(df_cal,ids,col_name="cal_sht45_temperature")

In [ ]:
plot_sensor_measurement(df_cal,ids,col_name="cal_gmp343_temperature")

In [ ]:
plot_sensor_measurement(df_cal,ids,col_name="cal_bme280_temperature")

In [ ]:
plot_sensor_measurement(df_cal,ids,col_name="cal_bme280_humidity")

In [ ]:
plot_sensor_measurement(df_cal,ids,col_name="cal_sht45_humidity")

In [ ]:
plot_sensor_measurement(df_cal,ids,col_name="cal_bme280_pressure")

In [ ]:
plot_sensor_measurement(df_cal,ids,col_name="cal_gmp343_filtered", cut_above=350, cut_below=470)

In [ ]:
plot_sensor_measurement(df_cal,ids,col_name="cal_gmp343_filtered", cut_above=470, cut_below=800)